In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

tf.enable_eager_execution()

In [2]:
x = tf.ones((2, 2)) # 创建一个所有元素都为1的张量

with tf.GradientTape() as t: # 使用梯度带API tf.GradientTape() 作为t
  t.watch(x)                # 观察变量x
  y = tf.reduce_sum(x)  # 求张量x的所有元素的总和 
  z = tf.multiply(y, y) 

In [3]:
print(z)

tf.Tensor(16.0, shape=(), dtype=float32)


In [4]:
print(x)
print(y)

tf.Tensor(
[[1. 1.]
 [1. 1.]], shape=(2, 2), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [5]:
dz_dx = t.gradient(z, x)  #  对z关于x求一阶导数， target = z， source = x
for i in [0, 1]:
  for j in [0, 1]:
    assert dz_dx[i][j].numpy() == 8.0

In [6]:
print(dz_dx)

tf.Tensor(
[[8. 8.]
 [8. 8.]], shape=(2, 2), dtype=float32)


In [7]:
x = tf.ones((2, 2))

with tf.GradientTape() as t:
  t.watch(x)
  y = tf.reduce_sum(x)
  z = tf.multiply(y, y)

# Use the tape to compute the derivative of z with respect to the
# intermediate value y.
dz_dy = t.gradient(z, y)
assert dz_dy.numpy() == 8.0 # 断言 若为false 就会报错

In [8]:
print(dz_dy)

tf.Tensor(8.0, shape=(), dtype=float32)


In [9]:
x = tf.constant(3.0)
with tf.GradientTape(persistent=True) as t:  #设置persistent=True 可以多次调用t.gradient()求导
  t.watch(x)
  y = x * x
  z = y * y
dz_dx = t.gradient(z, x)  # 108.0 (4*x^3 at x = 3)
dy_dx = t.gradient(y, x)  # 6.0  (2*x at x = 3)
print(dz_dx)
print(dy_dx)
del t  # Drop the reference to the tape

tf.Tensor(108.0, shape=(), dtype=float32)
tf.Tensor(6.0, shape=(), dtype=float32)


In [10]:
def f(x, y):
  output = 1.0
  for i in range(y):
    if i > 1 and i < 5:
      output = tf.multiply(output, x)
  return output

def grad(x, y):
  with tf.GradientTape() as t:
    t.watch(x)
    out = f(x, y)
    print(out)
  return t.gradient(out, x)

x = tf.convert_to_tensor(2.0)

assert grad(x, 6).numpy() == 12.0   # x^3   (3*x^2)
assert grad(x, 5).numpy() == 12.0  # x^3   (3*x^2)
assert grad(x, 4).numpy() == 4.0    # x^2  (2*x)

tf.Tensor(8.0, shape=(), dtype=float32)
tf.Tensor(8.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [11]:
print(x) # 将2.0 变成一个张量

tf.Tensor(2.0, shape=(), dtype=float32)


In [12]:
# 高阶求导
x = tf.Variable(1.0)  # Create a Tensorflow variable initialized to 1.0

with tf.GradientTape() as t:
  with tf.GradientTape() as t2:
    y = x * x * x
  # Compute the gradient inside the 't' context manager
  # which means the gradient computation is differentiable as well.
  dy_dx = t2.gradient(y, x)
d2y_dx2 = t.gradient(dy_dx, x)

assert dy_dx.numpy() == 3.0
assert d2y_dx2.numpy() == 6.0